### MID 12
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
### GRU model used, model type is 5 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import GRU, Dense
import tensorflow as tf
import csv

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

ant_1_3_scaled = combined_data[combined_data['version'] == 1.3] 
ant_1_4_scaled = combined_data[combined_data['version'] == 1.4] 

In [4]:
# Getting training and test data.
X_train = ant_1_3_scaled[cols_to_norm]
X_train = np.array(X_train)
Y_train = ant_1_3['bug']

X_test = ant_1_4_scaled[cols_to_norm]
X_test = np.array(X_test)
Y_test = ant_1_4['bug']

In [5]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [6]:
train_x 

array([[[0.14285714, 0.6       , 0.05      , ..., 0.16595094,
         0.08571429, 0.20908494]],

       [[0.18181818, 0.        , 0.025     , ..., 0.08097907,
         0.17142857, 0.26990307]],

       [[0.03896104, 0.2       , 0.        , ..., 0.08326661,
         0.02857143, 0.1095285 ]],

       ...,

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.07792208, 0.4       , 0.        , ..., 0.15132106,
         0.05714286, 0.16428454]]])

In [7]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [8]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [9]:
test_x

array([[[0.18181818, 0.4       , 0.        , ..., 0.12421366,
         0.11428571, 0.25815673]],

       [[0.06493506, 0.4       , 0.        , ..., 0.25748599,
         0.02857143, 0.13142763]],

       [[0.16883117, 0.        , 0.025     , ..., 0.03473548,
         0.02857143, 0.12636767]],

       ...,

       [[0.2987013 , 0.        , 0.        , ..., 0.2197236 ,
         0.68571429, 0.37856087]],

       [[0.14285714, 0.        , 0.2       , ..., 0.37338962,
         0.62857143, 0.76168885]],

       [[0.05194805, 0.2       , 0.        , ..., 0.06485188,
         0.14285714, 0.28749795]]])

In [10]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [11]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [12]:
# Designing and initializing the model.
model = Sequential()
model.add(GRU(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(GRU(80, dropout = 0.2, return_sequences=True))
model.add(GRU(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 1, 100)            36600     
_________________________________________________________________
gru_1 (GRU)                  (None, 1, 80)             43680     
_________________________________________________________________
gru_2 (GRU)                  (None, 60)                25560     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 105,901
Trainable params: 105,901
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 11ms/step - loss: 3.3376 - mse: 3.3376 - mae: 1.4696 - root_mean_squared_error: 1.8269 - mean_squared_logarithmic_error: 0.8214
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 2.9169 - mse: 2.9169 - mae: 1.3922 - root_mean_squared_error: 1.7079 - mean_squared_logarithmic_error: 0.6412
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 2.4237 - mse: 2.4237 - mae: 1.2902 - root_mean_squared_error: 1.5568 - mean_squared_logarithmic_error: 0.4775
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 1.8455 - mse: 1.8455 - mae: 1.1316 - root_mean_squared_error: 1.3585 - mean_squared_logarithmic_error: 0.3379
Epoch 5/100
4/4 [==============================] - 0s 14ms/step - loss: 1.3248 - mse: 1.3248 - mae: 0.9973 - root_mean_squared_error: 1.1510 - mean_squared_logarithmic_error: 0.2668
Epoch 6/100
4/4 [==============================] - 0s 14ms/step - loss: 1.2808 - mse: 1.28

4/4 [==============================] - 0s 14ms/step - loss: 0.9479 - mse: 0.9479 - mae: 0.8352 - root_mean_squared_error: 0.9736 - mean_squared_logarithmic_error: 0.2092
Epoch 46/100
4/4 [==============================] - 0s 13ms/step - loss: 0.9769 - mse: 0.9769 - mae: 0.8509 - root_mean_squared_error: 0.9884 - mean_squared_logarithmic_error: 0.2125
Epoch 47/100
4/4 [==============================] - 0s 13ms/step - loss: 0.9612 - mse: 0.9612 - mae: 0.8367 - root_mean_squared_error: 0.9804 - mean_squared_logarithmic_error: 0.2187
Epoch 48/100
4/4 [==============================] - 0s 14ms/step - loss: 0.9241 - mse: 0.9241 - mae: 0.7978 - root_mean_squared_error: 0.9613 - mean_squared_logarithmic_error: 0.2134
Epoch 49/100
4/4 [==============================] - 0s 15ms/step - loss: 0.9475 - mse: 0.9475 - mae: 0.8335 - root_mean_squared_error: 0.9734 - mean_squared_logarithmic_error: 0.2131
Epoch 50/100
4/4 [==============================] - 0s 11ms/step - loss: 0.9210 - mse: 0.9210 - ma

4/4 [==============================] - 0s 12ms/step - loss: 0.8927 - mse: 0.8927 - mae: 0.7801 - root_mean_squared_error: 0.9448 - mean_squared_logarithmic_error: 0.1980
Epoch 90/100
4/4 [==============================] - 0s 13ms/step - loss: 0.8585 - mse: 0.8585 - mae: 0.7692 - root_mean_squared_error: 0.9266 - mean_squared_logarithmic_error: 0.2005
Epoch 91/100
4/4 [==============================] - 0s 12ms/step - loss: 0.9700 - mse: 0.9700 - mae: 0.8259 - root_mean_squared_error: 0.9849 - mean_squared_logarithmic_error: 0.2214
Epoch 92/100
4/4 [==============================] - 0s 12ms/step - loss: 0.8999 - mse: 0.8999 - mae: 0.7894 - root_mean_squared_error: 0.9486 - mean_squared_logarithmic_error: 0.1921
Epoch 93/100
4/4 [==============================] - 0s 13ms/step - loss: 0.8879 - mse: 0.8879 - mae: 0.7943 - root_mean_squared_error: 0.9423 - mean_squared_logarithmic_error: 0.1894
Epoch 94/100
4/4 [==============================] - 0s 13ms/step - loss: 0.8644 - mse: 0.8644 - ma

In [16]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[1.3666266 ],
       [1.8865181 ],
       [0.8497438 ],
       [1.7090058 ],
       [1.7944719 ],
       [1.5567923 ],
       [1.2879735 ],
       [0.        ],
       [0.92508173],
       [2.041378  ],
       [1.4441104 ],
       [2.190116  ],
       [1.1146584 ],
       [1.4162484 ],
       [1.8047704 ],
       [0.        ],
       [2.1373188 ],
       [0.5697741 ],
       [0.        ],
       [1.0383091 ],
       [1.1617573 ],
       [0.6585943 ],
       [0.99351484],
       [1.5325365 ],
       [1.8258079 ],
       [1.5457364 ],
       [1.4994415 ],
       [1.0590466 ],
       [1.3703574 ],
       [0.        ],
       [0.        ],
       [0.44968998],
       [0.32263607],
       [1.521062  ],
       [0.35533065],
       [0.        ],
       [1.2592357 ],
       [0.        ],
       [1.0406712 ],
       [1.0775812 ],
       [1.3946483 ],
       [0.        ],
       [0.        ],
       [2.1506565 ],
       [0.        ],
       [0.08131059],
       [1.6190602 ],
       [1.782

In [17]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[1.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [0.],
       [1.],
       [2.],
       [1.],
       [2.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [2.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [1.],
       [0.],
       [2.],
       [0.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],

In [18]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [19]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([189.], dtype=float32)

In [20]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.48528624], dtype=float32)

In [21]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.49368343], dtype=float32)

In [22]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 4ms/step - loss: 1.3261 - mse: 1.3261 - mae: 0.9525 - root_mean_squared_error: 1.1516 - mean_squared_logarithmic_error: 0.4712
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [23]:
# Saving the model
model_id = 12
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model5/ant1.3_ant1.4_model5_1'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model5/ant1.3_ant1.4_model5_1/assets


In [24]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [25]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [26]:
score

[12,
 'ant-1.3',
 'ant-1.4',
 1.326117753982544,
 1.326117753982544,
 0.9524757266044617,
 1.1515718698501587,
 0.4711892604827881,
 0.48528623580932617,
 0.49368342757225037]

In [27]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [28]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)